Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat.

In [1]:
import pandas as pd 
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth

import matplotlib.pyplot as plt
from itertools import cycle

In [2]:
data = pd.read_csv('checkins.csv', low_memory=False)

data['latitude'] = data['latitude'].str.strip()
data['longitude'] = data['longitude'].str.strip()

data['latitude'].replace('', np.nan, inplace=True)
data['longitude'].replace('', np.nan, inplace=True)

data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [3]:
data.dropna(subset=['latitude', 'longitude'], inplace=True)
data['id'] = data['id'].astype('int32')
data['latitude'] = data['latitude'].astype('float64')
data['longitude'] = data['longitude'].astype('float64')
data['created_at'] = pd.to_datetime(data['created_at'])

print(data.shape)

data.dtypes

(396634, 6)


id                     int32
user_id              float64
venue_id             float64
latitude             float64
longitude            float64
created_at    datetime64[ns]
dtype: object

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [4]:
X = data[['latitude', 'longitude']].iloc[:100000]
X.shape

(100000, 2)

In [5]:
%%time

ms = MeanShift(bandwidth=0.1)
ms.fit(X.to_numpy())

labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("Number of estimated clusters : %d" % n_clusters_)

Number of estimated clusters : 3231
CPU times: user 3min 33s, sys: 1.35 s, total: 3min 35s
Wall time: 3min 35s


In [6]:
clusters = pd.DataFrame(ms.cluster_centers_)
clusters.columns = ['cluster_latitude', 'cluster_longitude']
clusters.head()

,cluster_latitude,cluster_longitude
0,40.717716,-73.991835
1,33.449438,-112.002140
2,33.446380,-111.901888
3,41.878244,-87.629843
4,37.688682,-122.409330


In [7]:
X['which_cluster'] = ms.predict(X)
X.head()

,latitude,longitude,which_cluster
1,38.895112,-77.036366,5
3,33.800745,-84.410520,7
7,45.523452,-122.676207,30
9,40.764462,-111.904565,65
10,33.448377,-112.074037,1


In [8]:
clusters['count'] = X.pivot_table(index = 'which_cluster', aggfunc = 'count', values = 'latitude')
clusters.head()

,cluster_latitude,cluster_longitude,count
0,40.717716,-73.991835,12506
1,33.449438,-112.002140,4692
2,33.446380,-111.901888,3994
3,41.878244,-87.629843,3363
4,37.688682,-122.409330,3526


Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров.

In [9]:
clusters = clusters[clusters['count'] > 15]
print(clusters.shape)
clusters.head()

(592, 3)


,cluster_latitude,cluster_longitude,count
0,40.717716,-73.991835,12506
1,33.449438,-112.002140,4692
2,33.446380,-111.901888,3994
3,41.878244,-87.629843,3363
4,37.688682,-122.409330,3526


In [10]:
for index, item in clusters.iterrows():
    print(str(item['cluster_latitude']) + ',' + str(item['cluster_longitude']))

40.71771639727507,-73.9918354198967
33.44943805020126,-112.00213969017547
33.44638027037988,-111.90188756212359
41.87824377967115,-87.62984336226329
37.68868157406161,-122.40933037359147
38.8861652155993,-77.04878333074303
33.357344562325096,-111.82265410760392
33.76663623218336,-84.39328918481657
42.363218639848895,-71.07368760857386
47.60624471741767,-122.33204382627093
36.117229142990276,-115.17107342280688
34.06039755458241,-118.24870902659876
44.97794782033687,-93.2673008852605
30.26718361698159,-97.74311928133027
40.76687624004169,-73.83335349045205
39.735830152625304,-104.98658042770822
39.95168037300773,-75.16273592391683
34.03548695312116,-118.43899771946148
32.98089338217789,-117.07811797821928
32.80302053531547,-96.76989743494408
37.34787114392445,-121.94728722354373
28.543501546632914,-81.37642862287088
32.71134443394278,-117.15363874791043
32.2217131518074,-110.92653515261165
34.12740221911527,-118.35188372510652
29.762697754725156,-95.3823137047244
43.040532816770465,-87.

![Clusters Distribution Map](map.png)

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

In [11]:
carnival_offices = [[33.751277, -118.188740],
                    [25.867736, -80.324116],
                    [51.503016, -0.075479],
                    [52.378894, 4.885084],
                    [39.366487, 117.036146],
                    [-33.868457, 151.205134 ]
                   ]   

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [12]:
def euclidian_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2)

def calculate_min_distance(lat, lon):
    min_distance = None
    for coords in carnival_offices:
        curr_dist = euclidian_distance(lat, lon, coords[0], coords[1])
        if (min_distance is None) or (curr_dist < min_distance):
            min_distance = curr_dist
    
    return min_distance

In [13]:
clusters['min_distance'] = clusters.apply(lambda row: calculate_min_distance(row['cluster_latitude'], row['cluster_longitude']), axis=1)
clusters.head()

,cluster_latitude,cluster_longitude,count,min_distance
0,40.717716,-73.991835,12506,16.143720
1,33.449438,-112.002140,4692,6.193959
2,33.446380,-111.901888,3994,6.294241
3,41.878244,-87.629843,3363,17.598580
4,37.688682,-122.409330,3526,5.772048


In [14]:
clusters = clusters.sort_values('min_distance')
clusters.head(20)

,cluster_latitude,cluster_longitude,count,min_distance
420,-33.860630,151.204776,28,0.007835
370,52.372964,4.892317,31,0.009353
419,25.845672,-80.318891,28,0.022674
58,51.502991,-0.125537,254,0.050058
51,33.809878,-118.148924,281,0.070848
29,25.785812,-80.217938,564,0.134109
167,25.705350,-80.283429,80,0.167406
92,26.010098,-80.199991,138,0.188876
87,33.888325,-118.048928,100,0.195779
42,33.872986,-118.362091,384,0.211811


In [15]:
def write_answer(lat, lon):
    with open('submission_MeanShift_1.txt', 'w') as fout:
        fout.write(f'{lat} {lon}')
        fout.close()

# Nearest cluster is (-33.860630, 151.204776) adjacent to -33.868457, 151.205134 (Sydney Office)

nearest_cluster = clusters[clusters['min_distance'] == clusters['min_distance'].min()].values[0]
print(nearest_cluster)

write_answer(nearest_cluster[0], nearest_cluster[1])

[-3.38606304e+01  1.51204776e+02  2.80000000e+01  7.83475816e-03]
